In [ ]:
# -*- coding: utf-8 -*-
import os
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain.vectorstores import Chroma
import dotenv


In [2]:

# Load environment variables
dotenv.load_dotenv()

# Initialize models
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

os.environ['OPENAI_API_KEY'] = api_key
embeddings_model = OpenAIEmbeddings(model="text-embedding-3-large")
llm = ChatOpenAI(model="gpt-4o-mini")

# Load and process CSV files
print("Loading CSV data...")
loaders = [
    CSVLoader(file_path="/home/omen/Sanju/Valuehealth/QA/Data/Diseaseprecaution.csv"),
    CSVLoader(file_path="/home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv")
]
documents = []
for loader in loaders:
    documents.extend(loader.load())

# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)

# Create vector store
print("Indexing documents...")
vector_store = Chroma.from_documents(
    # documents=all_splits,
    embedding=embeddings_model,
    persist_directory="./chroma_db"
)


Loading CSV data...
Indexing documents...


In [3]:
all_splits

[Document(metadata={'source': '/home/omen/Sanju/Valuehealth/QA/Data/Diseaseprecaution.csv', 'row': 0}, page_content='Disease: Drug Reaction\nPrecaution_1: stop irritation\nPrecaution_2: consult nearest hospital\nPrecaution_3: stop taking drug\nPrecaution_4: follow up'),
 Document(metadata={'source': '/home/omen/Sanju/Valuehealth/QA/Data/Diseaseprecaution.csv', 'row': 1}, page_content='Disease: Malaria\nPrecaution_1: Consult nearest hospital\nPrecaution_2: avoid oily food\nPrecaution_3: avoid non veg food\nPrecaution_4: keep mosquitos out'),
 Document(metadata={'source': '/home/omen/Sanju/Valuehealth/QA/Data/Diseaseprecaution.csv', 'row': 2}, page_content='Disease: Allergy\nPrecaution_1: apply calamine\nPrecaution_2: cover area with bandage\nPrecaution_3: \nPrecaution_4: use ice to compress itching'),
 Document(metadata={'source': '/home/omen/Sanju/Valuehealth/QA/Data/Diseaseprecaution.csv', 'row': 3}, page_content='Disease: Hypothyroidism\nPrecaution_1: reduce stress\nPrecaution_2: exe

In [6]:

# Create conversation chain
qa_chain = ConversationalRetrievalChain.from_llm(
    llm,
    vector_store.as_retriever(search_kwargs={'k': 3}),
    return_source_documents=True
)


In [13]:

# Chat interface with explicit history formatting
def chat_with_csv():
    chat_history = []
    print("Medical CSV Chatbot - Type 'exit' to end")
    
    while True:
        query = input("\nQuestion: ")
        if query.lower() in ['exit', 'quit']:
            break
            
        # Format previous history into string
        history_str = "\n".join([f"Q: {q}\nA: {a}" for q, a in chat_history])
        formatted_query = (
            "{prev_queries_answers: '''" + history_str + "''', "
            f"new_query: '''{query}'''}}"
        )
        
        # Get response with proper input format
        result = qa_chain.invoke({
            "question": formatted_query,
            "chat_history": chat_history  # Add this line
        })
        
        answer = result['answer']
        
        # Display results
        print(f"\nAnswer: {answer}")
        print("\nSources:")
        for doc in result['source_documents']:
            source = doc.metadata.get('source', 'Unknown CSV')
            print(f"- {source}")
        
        # Update history with original query and answer
        chat_history.append((query, answer))


In [14]:
chat_with_csv()

Medical CSV Chatbot - Type 'exit' to end

Answer: Hello! How can I assist you today?

Sources:
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv

Answer: I don't know.

Sources:
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv

Answer: I don't know.

Sources:
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv

Answer: The symptoms of drug reactions can include:

1. Itching
2. Skin rash
3. Stomach pain
4. Burning micturition

These are some of the symptoms associated with drug reactions.

Sources:
- /home/omen/Sanju/Valuehealth/QA/Data/DiseaseAndSymptoms.csv
- /home/omen/Sanju/Valueheal